# Imports

In [1]:
import warnings

import pandas as pd

from functions.loading import load_data

from functions.training_pipeline import training_pipeline
from functions.models import xgboost_model, catboost_model, lgbm_model

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

# Parameters definition

In [2]:
# Training parameters
restricted_features=False
save=True
customized_model=True

models = {
        # "xgboost": xgboost_model,
        "catboost": catboost_model,
        # "lgbm": lgbm_model,
}

targets = ["cf1", "cf2", "cf3", "cf123"]

# Results containers
summary_final=[]
Summary_Final_train = []
summary_metrics_detailed = pd.DataFrame()
estimated_scopes = []

# Data Loading 

In [3]:
preprocessed_dataset = load_data(save=True)

# Training pipeline 

In [4]:
best_scores, best_stds, summary_global, summary_metrics_detailed = training_pipeline(
    targets=targets,
    models=models,
    summary_final=summary_final,
    summary_metrics_detailed=summary_metrics_detailed,
    estimated_scopes=estimated_scopes,
    preprocessed_dataset=preprocessed_dataset,
    restricted_features=restricted_features,
    save=save,
    customized_model=customized_model,    
)

INFO:functions.training_pipeline:Training for target: cf1


Using pre created preprocessed files


INFO:functions.training_pipeline:Preprocessing done
INFO:functions.training_pipeline:Training model: catboost
INFO:functions.training_pipeline:Modelisation done
INFO:functions.training_pipeline:Elapsed time for target cf1: 29.83 seconds


[0.5403244285897988]
[0.5403244285897988]


INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:functions.training_pipeline:Training for target: cf2


Using pre created preprocessed files


INFO:functions.training_pipeline:Preprocessing done
INFO:functions.training_pipeline:Training model: catboost
INFO:functions.training_pipeline:Modelisation done
INFO:functions.training_pipeline:Elapsed time for target cf2: 28.02 seconds


[0.43016517575893726]
[0.43016517575893726]


INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.


In [ ]:
best_scores

[0.5403244285897988,
 0.43016517575893726,
 0.9038417992384329,
 0.5552884023779382]

In [ ]:
best_stds

[0.10410354550253634,
 0.07483751852769431,
 0.08653843381578383,
 0.05204548223712658]

# Model application

In [ ]:
from functions.apply_model import apply_model_on_raw_data

In [ ]:
raw_dataset = load_data(filter_outliers=False, save=False)
estimations = apply_model_on_raw_data(raw_dataset,
    save=False,
    restricted_features=restricted_features,
)
estimations

ValueError: Expected a 1D array, got an array with shape (596977, 2)